In [2]:
elastic_rods_dir = '../'
weaving_dir = '../../../Weaving/'
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt

In [3]:
# Reloadable libraries
import finite_diff, importlib
importlib.reload(finite_diff)
from finite_diff import fd_hessian_test, fd_gradient_test, gradient_convergence_plot, hessian_convergence_plot, hessian_convergence_block_plot

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (initialize_linkage, 
                                   update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

In [51]:
# Sphere Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1, 10]
ISO_CS = [4, 4]
MODEL_PATH = osp.join(weaving_dir + 'models/equal_edge_atI.obj')
SURFACE_PATH = osp.join(weaving_dir + 'surface_models/centered_sphere_100mm.obj')
SUBDIVISION_RESOLUTION = 5
REGULARIZATION_WEIGHT = 0.05
SMOOTHING_WEIGHT = 0

In [29]:
# Sphere to simple hemoglobin sequence
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [0.01, 0.5]
ISO_CS = [0.04, 0.04]
MODEL_PATH = osp.join(weaving_dir + 'models/simple_sphere_to_hemoglobin/5.obj')
SUBDIVISION_RESOLUTION = 20
REGULARIZATION_WEIGHT = 0.001
SMOOTHING_WEIGHT = 0
MODEL_NAME = './results/simple_sphere_to_hemoglobin'
RIBBON_NAME = 'simple_hemoglobin_strip'

In [55]:
linkage = elastic_rods.SurfaceAttractedLinkage(SURFACE_PATH, True, MODEL_PATH, SUBDIVISION_RESOLUTION, False)
linkage.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, RIBBON_CS, stiffAxis=elastic_rods.StiffAxis.D1))
linkage.updateSourceFrame()
linkage.set_design_parameter_config(True, True)
linkage.attraction_weight = 100
linkage.set_holdClosestPointsFixed(False)
linkage.set_attraction_tgt_joint_weight(0.01)
with so():
    opts = elastic_rods.NewtonOptimizerOptions()
    opts.niter = 400
    opts.verbose = 10
    elastic_rods.compute_equilibrium(linkage, options = opts)

In [56]:
with so():
    opts = elastic_rods.NewtonOptimizerOptions()
    opts.niter = 400
    opts.verbose = 10
    elastic_rods.compute_equilibrium(linkage, options = opts)

In [62]:
linkage.attraction_weight = 1
import compute_vibrational_modes
lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(linkage, fixedVars=[], mtype=compute_vibrational_modes.MassMatrixType.FULL, n=16, sigma=-1e-6)

import mode_viewer, importlib
importlib.reload(mode_viewer);
mview = mode_viewer.ModeViewer(linkage, modes, lambdas, amplitude=1.0)
mview.show()

m: 2430  nnz: 67410


In [16]:
def initialize_curved_linkage(cross_section = ISO_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH, cam_param = default_camera_parameters, regularization_weight = 0.1, smoothing_weight = 1):
    l = elastic_rods.RodLinkage(model_path, subdivision_res, False)
    driver=0
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    l.set_design_parameter_config(use_restLen = True, use_restKappa = True)
#     design_opts = elastic_rods.NewtonOptimizerOptions()
#     design_opts.niter = 500
#     design_opts.verbose = 10
#     elastic_rods.designParameter_solve(l, design_opts, regularization_weight = regularization_weight, smoothing_weight = smoothing_weight)
    jdo = l.dofOffsetForJoint(driver)
    fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
    eqm_opts = elastic_rods.NewtonOptimizerOptions()
    eqm_opts.niter = 400
    elastic_rods.compute_equilibrium(l, fixedVars=fixedVars, options = eqm_opts)
    view = linkage_vis.LinkageViewer(l, width=1024, height=640)
    view.setCameraParams(cam_param)
    return l, view

In [17]:
with so(): normal_linkage, view = initialize_curved_linkage(model_path = MODEL_PATH, cam_param = default_camera_parameters, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, regularization_weight = REGULARIZATION_WEIGHT, smoothing_weight=SMOOTHING_WEIGHT)

In [33]:
driver = 0
jdo = normal_linkage.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint

import compute_vibrational_modes
lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(normal_linkage, fixedVars=fixedVars, mtype=compute_vibrational_modes.MassMatrixType.FULL, n=16, sigma=-1e-6)

import mode_viewer, importlib
importlib.reload(mode_viewer);
mview = mode_viewer.ModeViewer(normal_linkage, modes, lambdas, amplitude=1.0)
mview.show()

m: 2424  nnz: 66954
